# Inbound Auth

AgentCore Identity lets you validate inbound access (Inbound Auth) for users and applications calling agents or tools in an AgentCore Runtime or validate access to AgentCore Gateway targets. It also provide secure outbound access (Outbound Auth) from an agent to external services or a Gateway target. It integrates with your existing identity providers (such as Amazon Cognito) while enforcing permission boundaries for agents acting independently or on behalf of users (via OAuth).

Inbound Auth validates callers attempting to invoke agents or tools, whether they're hosted in AgentCore Runtime, AgentCore Gateway , or in other environments. Inbound Auth works with IAM (SigV4 credentials) or with OAuth authorization.

By default, Amazon Bedrock AgentCore uses IAM credentials, meaning user requests to the agent are authenticated with the user's IAM credentials. If you use OAuth, you will need to specify the following when configuring your AgentCore Runtime resources or AgentCore Gateway endpoints:

- OAuth discovery server Url — A string that must match the pattern ^.+/\.well-known/openid-configuration$ for OpenID Connect discovery URLs

- Allowed audiences — List of allowed audiences for JWT tokens

- Allowed clients — List of allowed client identifiers

If you use the AgentCore CLI, you can specify the type of authorization (and OAuth discovery server) for an AgentCore Runtime when you use the **configure** command. You can also use the CreateAgentRuntime operation and Amazon Bedrock AgentCore console. If you are creating a Gateway, you use the CreateGateway operation, or the console.

Before the user can use the agent, the client application must have the user authenticate with the OAuth authorizer. Your client receives a bearer token which it then passes to the agent in an invocation request. Upon receipt the agent validates the token with the authorization server before allowing access.


## Overview

In this tutorial we will modify the agent you deployed in 01-AgentCore-runtime and configure it for Inbound Auth using Cognito as the Identity provider. You will set up a Cognito User pool with one user and an app client. You will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime with Inbound Auth using the Cognito user pool. 

### Tutorial Architecture

<div style="text-align:center">
    <img src="images/inbound_auth_cognito.png" width="90%"/>
</div>

### Tutorial Details


| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                                   |
| Agent type          | Single                                                                           |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Haiku 4.5                                                      |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands Agent and Amazon Bedrock Model |
| Tutorial vertical   | Cross-vertical                                                                   |
| Example complexity  | Easy                                                                             |
| Inbound Auth        | Cognito                                                                          |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                                     |



### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime with Inbound Auth using Amazon Cognito
* Using Amazon Bedrock models
* Using Strands Agents


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker running

In [1]:
!pip install --force-reinstall -U -r requirements.txt --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.4.0 requires nvidia-ml-py3<8.0,>=7.352.0, which is not installed.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
jupyter-ai 2.31.6 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
sagemaker-studio 1.1.1 requires pydynamodb>=0.7.4, which is not installed.
aiobotocore 2.22.0 requires botocore<1.37.4,>=1.37.2, but you have botocore 1.42.9 which is incompatible.
amazon-sagemaker-jupyter-ai-q-developer 1.2.8 requires numpy<=2.0.1, but you have numpy 2.3.5 which is incompatible.
amazon-sagemaker-sql-magic 0.1.4 requires numpy<2, but you have numpy 2.3.5 which is incompatible.
autogluon-multimodal 1.4.0 requires j

## Setting up Amazon Cognito for Authentication

Lets provision a Cognito Userpool with an App client and one test user. We'll use Amazon Cognito to provide JWT tokens for accessing our deployed MCP server. To do so, we will use the `setup_cognito_user_pool` support function from our `utils` script.

Note: The Cognito access_token is valid for 2 hours only. If the access_token expires you can vend another access_token by using the `reauthenticate_user` method.

In [2]:
import sys
import os

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from utils import setup_cognito_user_pool, reauthenticate_user

sys.path[0]: /home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples


In [3]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool()
print("Cognito setup completed ✓")

Setting up Amazon Cognito user pool...
Pool id: us-west-2_jpbCRPpT4
Discovery URL: https://cognito-idp.us-west-2.amazonaws.com/us-west-2_jpbCRPpT4/.well-known/openid-configuration
Client ID: 57h2c0e8p2n1k9icp9dk7581ll
Bearer Token: eyJraWQiOiJjTGFFTnoxN2RRV3ZRd1hVK09pUG50b0pCaGhDcUJkNlwvdUpsV2kwMWhQMD0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiJiOGYxODM5MC1hMGUxLTcwYWEtNWEyZS1hZDUzNzIyNjk4YWMiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtd2VzdC0yLmFtYXpvbmF3cy5jb21cL3VzLXdlc3QtMl9qcGJDUlBwVDQiLCJjbGllbnRfaWQiOiI1N2gyYzBlOHAybjFrOWljcDlkazc1ODFsbCIsIm9yaWdpbl9qdGkiOiIwNmM0NWMwNi03NjdiLTQxMjQtYjk0OC1lZTE4NGQ2MGI5NTQiLCJldmVudF9pZCI6IjcyNjJjMGI5LTQ4MGEtNDA1OC1hOTUyLTBmNjNmM2IyNGI2NiIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE3NjU2MzUzODUsImV4cCI6MTc2NTYzODk4NSwiaWF0IjoxNzY1NjM1Mzg1LCJqdGkiOiI3OTY2ZWM3MS0yOGExLTRjYTUtYTkyZC00ZDYxMjkxMDgwYTAiLCJ1c2VybmFtZSI6InRlc3R1c2VyIn0.h5SOBKGgK0fjdp1wClBDhm4-sQNfDy_trq0h76tg_O4by__BLgNuHddwe_638RpcAQ75NBhrvz0

## Preparing your agent for deployment on AgentCore Runtime

### Strands Agents with Amazon Bedrock model
Let's start with our Strands Agent we created in the 01-AgentCore-runtime tutorial and configure it with Inbound Auth that uses Amazon Cognito as the Identity Provider.

In [4]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel

app = BedrockAgentCoreApp()

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"


model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

@app.entrypoint
def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

Writing strands_claude.py


## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCode Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Configure AgentCore Runtime deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

**Important** - Update the Cognito Discovery url and the Cognito App client id from the previous steps.

In [5]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

discovery_url = cognito_config.get("discovery_url")

client_id = cognito_config.get("client_id")

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="strands_agent_inbound_identity",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "discoveryUrl": discovery_url,
            "allowedClients": [client_id]
        }
    }
)
response


Entrypoint parsed: file=/home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/13-AgentCore-identity/03-Inbound Auth example/strands_claude.py, bedrock_agentcore_name=strands_claude
Memory disabled - agent will be stateless
Configuring BedrockAgentCore agent: strands_agent_inbound_identity
Memory disabled
Network mode: PUBLIC


⚠️ Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64', so local builds
won't work.
Please use default launch command which will do a remote cross-platform build using code build.For deployment other
options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: Dockerfile
Generated .dockerignore: /home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/13-AgentCore-identity/03-Inbound Auth example/.dockerignore
Setting 'strands_agent_inbound_identity' as default agent
Bedrock AgentCore configured: /home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/13-AgentCore-identity/03-Inbound Auth example/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/13-AgentCore-identity/03-Inbound Auth example/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/13-AgentCore-identity/03-Inbound Auth example/Dockerfile'), dockerignore_path=PosixPath('/home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/13-AgentCore-identity/03-Inbound Auth example/.dockerignore'), runtime='Docker', runtime_type=None, region='us-west-2', account_id='455933813601', execution_role=None, ecr_repository=None, auto_create_ecr=True, s3_path=None, auto_create_s3=False, memory_id=None, network_mode='PUBLIC', network_subnets=None, network_security_groups=None, network_vpc_id=None)

## Review the AgentCore configuration

In [6]:
!cat .bedrock_agentcore.yaml

default_agent: strands_agent_inbound_identity
agents:
  strands_agent_inbound_identity:
    name: strands_agent_inbound_identity
    entrypoint: /home/sagemaker-user/amazon-bedrock-agentcore-workshop/strand-agent-samples/13-AgentCore-identity/03-Inbound
      Auth example/strands_claude.py
    deployment_type: container
    runtime_type: null
    platform: linux/arm64
    container_runtime: docker
    source_path: null
    aws:
      execution_role: null
      execution_role_auto_create: true
      account: '455933813601'
      region: us-west-2
      ecr_repository: null
      ecr_auto_create: true
      s3_path: null
      s3_auto_create: false
      network_configuration:
        network_mode: PUBLIC
        network_mode_config: null
      protocol_configuration:
        server_protocol: HTTP
      observability:
        enabled: true
      lifecycle_configuration:
        idle_runtime_session_timeout: null
        max_lifetime: null
    bedrock_agentcore:
      agent_id: null
     

In [7]:
cat Dockerfile

FROM ghcr.io/astral-sh/uv:python3.12-bookworm-slim
WORKDIR /app

# All environment variables in one layer
ENV UV_SYSTEM_PYTHON=1 \
    UV_COMPILE_BYTECODE=1 \
    UV_NO_PROGRESS=1 \
    PYTHONUNBUFFERED=1 \
    DOCKER_CONTAINER=1 \
    AWS_REGION=us-west-2 \
    AWS_DEFAULT_REGION=us-west-2



COPY requirements.txt requirements.txt
# Install from requirements file
RUN uv pip install -r requirements.txt




RUN uv pip install aws-opentelemetry-distro==0.12.2


# Signal that this is running in Docker for host binding logic
ENV DOCKER_CONTAINER=1

# Create non-root user
RUN useradd -m -u 1000 bedrock_agentcore
USER bedrock_agentcore

EXPOSE 9000
EXPOSE 8000
EXPOSE 8080

# Copy entire project (respecting .dockerignore)
COPY . .

# Use the full module path

CMD ["opentelemetry-instrument", "python", "-m", "strands_claude"]


### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [8]:
launch_result = agentcore_runtime.launch()
launch_result

🚀 Launching Bedrock AgentCore (cloud mode - RECOMMENDED)...
   • Deploy Python code directly to runtime
   • No Docker required (DEFAULT behavior)
   • Production-ready deployment

💡 Deployment options:
   • runtime.launch()                → Cloud (current)
   • runtime.launch(local=True)      → Local development
Memory disabled - skipping memory creation
Starting CodeBuild ARM64 deployment for agent 'strands_agent_inbound_identity' to account 455933813601 (us-west-2)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: strands_agent_inbound_identity
ECR repository available: 455933813601.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_agent_inbound_identity
Getting or creating execution role for agent: strands_agent_inbound_identity
Using AWS region: us-west-2, account ID: 455933813601
Role name: AmazonBedrockAgentCoreSDKRuntime-us-west-2-2213d1079e


Repository doesn't exist, creating new ECR repository: bedrock-agentcore-strands_agent_inbound_identity


Role doesn't exist, creating new execution role: AmazonBedrockAgentCoreSDKRuntime-us-west-2-2213d1079e
Starting execution role creation process for agent: strands_agent_inbound_identity
✓ Role creating: AmazonBedrockAgentCoreSDKRuntime-us-west-2-2213d1079e
Creating IAM role: AmazonBedrockAgentCoreSDKRuntime-us-west-2-2213d1079e
✓ Role created: arn:aws:iam::455933813601:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-2213d1079e
✓ Execution policy attached: BedrockAgentCoreRuntimeExecutionPolicy-strands_agent_inbound_identity
Role creation complete and ready for use with Bedrock AgentCore
Execution role available: arn:aws:iam::455933813601:role/AmazonBedrockAgentCoreSDKRuntime-us-west-2-2213d1079e
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: strands_agent_inbound_identity
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-west-2-2213d1079e
CodeBuild role doesn't exist, creating new role: AmazonBedrockAgentCoreSDKCodeBuild-us-we

LaunchResult(mode='codebuild', tag='bedrock_agentcore-strands_agent_inbound_identity:latest', env_vars=None, port=None, runtime=None, ecr_uri='455933813601.dkr.ecr.us-west-2.amazonaws.com/bedrock-agentcore-strands_agent_inbound_identity', agent_id='strands_agent_inbound_identity-tCUPp27omP', agent_arn='arn:aws:bedrock-agentcore:us-west-2:455933813601:runtime/strands_agent_inbound_identity-tCUPp27omP', codebuild_id='bedrock-agentcore-strands_agent_inbound_identity-builder:776fdc89-ea8a-4f43-b08b-ad745f65f035', build_output=None)

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [9]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: strands_agent_inbound_identity


'READY'

### Invoking AgentCore Runtime without authorization

Finally, we can invoke our AgentCore Runtime with a payload. Try running the following cell and you will see an error that says **"AccessDeniedException: An error occurred (AccessDeniedException) when calling the InvokeAgentRuntime operation: Agent is configured for a different authorization token type".**

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [10]:
invoke_response = agentcore_runtime.invoke({"prompt": "How is the weather now?"})
invoke_response

AccessDeniedException: An error occurred (AccessDeniedException) when calling the InvokeAgentRuntime operation: Authorization method mismatch. The agent is configured for a different authorization method than what was used in your request. Check the agent's authorization configuration and ensure your request uses the matching method (OAuth or SigV4)

### Invoking AgentCore Runtime with authorization

Lets invoke the agent with the right authorization token type. In our case, it will be the Cognito access token. Copy the access token from the cell "**Provision a Cognito User Pool**"

In [11]:
bearer_token = reauthenticate_user(cognito_config.get("client_id"))
invoke_response = agentcore_runtime.invoke(
    {"prompt": "How is the weather now?"}, 
    bearer_token=bearer_token
)
invoke_response

Using JWT authentication


{'response': '"The weather is currently **sunny**! It\'s a nice day outside. 🌞"'}

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [12]:
from boto3.session import Session
import boto3
boto_session = Session()

agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# Congratulations!